# Phase 1: News Sources Exploration

This notebook explores different news sources and checks if our current dataset already contains news data.

In [13]:
import sys
from pathlib import Path

project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import os
os.chdir(project_root)

In [14]:
import pandas as pd
import numpy as np
from src.data.loader import load_raw_dataset
from src.utils.config import load_config

## 1.1 Check Current Dataset for News Data

In [15]:
# Load current dataset
config = load_config()
print(f"Dataset name: {config.data.dataset_name}")

# Check local file first
local_file = Path("data/raw/sp500_stocks_data.parquet")
if local_file.exists():
    print(f"\nLoading local dataset: {local_file}")
    df = pd.read_parquet(local_file)
    print(f"Shape: {df.shape}")
    print(f"\nColumns ({len(df.columns)}):")
    for i, col in enumerate(df.columns, 1):
        print(f"  {i}. {col}")
    
    # Check for news-related columns
    news_keywords = ['news', 'article', 'headline', 'title', 'content', 'text', 'sentiment']
    news_cols = [col for col in df.columns if any(kw in col.lower() for kw in news_keywords)]
    
    if news_cols:
        print(f"\n✅ Found news-related columns: {news_cols}")
        for col in news_cols:
            print(f"\n  {col}:")
            print(f"    Non-null count: {df[col].notna().sum()} / {len(df)}")
            if df[col].dtype == 'object':
                sample = df[col].dropna().iloc[0] if df[col].notna().any() else None
                if sample:
                    print(f"    Sample (first 200 chars): {str(sample)[:200]}...")
    else:
        print(f"\n❌ No news-related columns found in dataset")
    
    # Show sample row
    print(f"\nSample row (first 5 columns):")
    print(df.iloc[0][df.columns[:5]])
else:
    print(f"\n⚠️  Local file not found. Would need to load from Hugging Face.")

Dataset name: pmoe7/SP_500_Stocks_Data-ratios_news_price_10_yrs

⚠️  Local file not found. Would need to load from Hugging Face.


## 1.2 Check Hugging Face Dataset Structure

### Option 4: FinBERT for Text Processing

FinBERT is a specialized BERT model trained on financial texts. It's perfect for:
- **Text embeddings** - Converting news text to numerical vectors
- **Sentiment analysis** - Classifying news as positive/negative/neutral
- **Financial domain understanding** - Better than general BERT for financial texts

In [16]:
# FinBERT - Financial BERT model
# Hugging Face: ProsusAI/finbert
# Perfect for financial news embeddings and sentiment analysis

def test_finbert():
    """Test FinBERT for financial text processing"""
    try:
        from transformers import AutoTokenizer, AutoModelForSequenceClassification
        import torch
        
        print("Loading FinBERT model...")
        print("This may take a minute on first run (downloading ~500MB)...")
        
        # Load FinBERT model and tokenizer
        model_name = "ProsusAI/finbert"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name)
        model.eval()
        
        print(f"✅ FinBERT loaded successfully!")
        print(f"   Model: {model_name}")
        print(f"   Max sequence length: {tokenizer.model_max_length}")
        
        # Test with sample financial news
        test_texts = [
            "Apple Inc. reported strong quarterly earnings, beating analyst expectations.",
            "The stock market crashed following the unexpected interest rate hike.",
            "Microsoft announced a new partnership with major cloud providers.",
        ]
        
        print(f"\\nTesting sentiment analysis on sample texts:")
        print("=" * 80)
        
        for text in test_texts:
            # Tokenize
            inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
            
            # Get predictions
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
            
            # Get sentiment labels
            labels = ["positive", "negative", "neutral"]
            scores = predictions[0].tolist()
            
            # Find predicted label
            predicted_idx = scores.index(max(scores))
            predicted_label = labels[predicted_idx]
            confidence = scores[predicted_idx]
            
            print(f"\\nText: {text[:70]}...")
            print(f"Sentiment: {predicted_label.upper()} (confidence: {confidence:.3f})")
            print(f"Scores: positive={scores[0]:.3f}, negative={scores[1]:.3f}, neutral={scores[2]:.3f}")
        
        print("\\n" + "=" * 80)
        print("\\n✅ FinBERT is working correctly!")
        print("\\nUsage:")
        print("  1. For sentiment: Use the classification head (as shown above)")
        print("  2. For embeddings: Use model.bert() to get hidden states")
        print("  3. For fine-tuning: Can further train on your specific data")
        
        return model, tokenizer
        
    except ImportError:
        print("❌ transformers library not installed")
        print("Install with: pip install transformers torch")
        return None, None
    except Exception as e:
        print(f"❌ Error loading FinBERT: {e}")
        return None, None

print("Testing FinBERT...")
finbert_model, finbert_tokenizer = test_finbert()

Testing FinBERT...
Loading FinBERT model...
This may take a minute on first run (downloading ~500MB)...
✅ FinBERT loaded successfully!
   Model: ProsusAI/finbert
   Max sequence length: 512
\nTesting sentiment analysis on sample texts:
\nText: Apple Inc. reported strong quarterly earnings, beating analyst expecta...
Sentiment: POSITIVE (confidence: 0.952)
Scores: positive=0.952, negative=0.025, neutral=0.022
\nText: The stock market crashed following the unexpected interest rate hike....
Sentiment: NEGATIVE (confidence: 0.909)
Scores: positive=0.017, negative=0.909, neutral=0.075
\nText: Microsoft announced a new partnership with major cloud providers....
Sentiment: POSITIVE (confidence: 0.920)
Scores: positive=0.920, negative=0.010, neutral=0.070
\n================================================================================
\n✅ FinBERT is working correctly!
\nUsage:
  1. For sentiment: Use the classification head (as shown above)
  2. For embeddings: Use model.bert() to get hidden

## FinBERT Usage: Embeddings vs Sentiment

FinBERT може да се използва по **два начина**:

1. **Sentiment Analysis (оценка)** - Класифицира новината като positive/negative/neutral
2. **Text Embeddings (като features)** - Конвертира текста в числов вектор за невронната мрежа

**За нашия модел ще използваме EMBEDDINGS като features!**

In [ ]:
# Example: Using FinBERT embeddings as features for the model

from src.models.news_encoder import FinBERTEncoder
import numpy as np

print("=" * 80)
print("FinBERT: Embeddings vs Sentiment")
print("=" * 80)

# Initialize encoder
encoder = FinBERTEncoder()

# Sample news text
sample_news = [
    "Apple Inc. reported strong quarterly earnings, beating analyst expectations by 15%.",
    "The Federal Reserve announced an unexpected interest rate hike, causing market volatility.",
    "Microsoft announced a new strategic partnership with major cloud providers.",
]

print("\\n1. SENTIMENT ANALYSIS (оценка):")
print("-" * 80)
for news in sample_news:
    sentiment = encoder.get_sentiment(news)
    print(f"\\nNews: {news[:60]}...")
    print(f"Sentiment: {sentiment['label'].upper()}")
    print(f"Probabilities: {sentiment['probabilities']}")

print("\\n\\n2. TEXT EMBEDDINGS (като features за модела):")
print("-" * 80)

# Get embeddings (these will be used as features in the neural network)
embeddings = encoder.encode_text(sample_news)
print(f"\\nEmbeddings shape: {embeddings.shape}")
print(f"  - {embeddings.shape[0]} news items")
print(f"  - {embeddings.shape[1]} dimensions per embedding")
print(f"\\nThis is what we'll feed into the neural network!")

# Show sample embedding
print(f"\\nSample embedding (first 10 dimensions):")
print(embeddings[0][:10])

print("\\n\\n3. COMBINED APPROACH:")
print("-" * 80)
print("We can use BOTH:")
print("  - Embeddings → Input features for the model")
print("  - Sentiment → Additional feature or for analysis")

# Get both embeddings and sentiment
embeddings_with_sentiment, sentiments = encoder.encode_text(
    sample_news,
    return_sentiment=True
)

print(f"\\nEmbeddings: {embeddings_with_sentiment.shape}")
print(f"Sentiments: {len(sentiments)} items")

# We could add sentiment score as an additional feature
sentiment_scores = []
for sent in sentiments:
    # Convert sentiment to numerical score
    # positive = +1, neutral = 0, negative = -1
    score_map = {'positive': 1.0, 'neutral': 0.0, 'negative': -1.0}
    score = score_map.get(sent['label'], 0.0)
    sentiment_scores.append(score)

sentiment_scores = np.array(sentiment_scores)
print(f"\\nSentiment scores: {sentiment_scores}")
print(f"  (Can be added as additional feature alongside embeddings)")

print("\\n" + "=" * 80)
print("CONCLUSION:")
print("=" * 80)
print("""
✅ FinBERT embeddings → Main features for neural network (768 dimensions)
✅ Sentiment scores → Optional additional feature (1 dimension)

The embeddings capture semantic meaning of the news text,
which the model can learn to correlate with price movements.
""")

## Важно: Откъде идват новините?

**FinBERT НЕ извлича новини!** FinBERT само обработва новини които ние му даваме.

**Пълният pipeline е:**

```
┌─────────────────┐
│  News Source    │  ← Извлича новини от интернет
│  (yfinance/     │     (yfinance, Alpha Vantage, и т.н.)
│   Alpha Vantage)│
└────────┬────────┘
         │
         │ новини (текст)
         ▼
┌─────────────────┐
│    FinBERT      │  ← Обработва новините
│  (news_encoder) │     (embeddings + sentiment)
└────────┬────────┘
         │
         │ embeddings (768 числа)
         ▼
┌─────────────────┐
│  Our Model      │  ← Използва embeddings като features
│  (Transformer)  │
└─────────────────┘
```

**Стъпки:**
1. **News Source** извлича новини → "Apple reported strong earnings..."
2. **FinBERT** обработва новините → [0.23, -0.45, ..., 0.12] (768 числа)
3. **Нашият модел** използва embeddings → прогноза за цената

In [ ]:
# Пример: Пълен pipeline от новини до embeddings

print("=" * 80)
print("ПЪЛЕН PIPELINE: News Source → FinBERT → Embeddings")
print("=" * 80)

# Стъпка 1: Извличане на новини (от news source)
print("\\n1. ИЗВЛИЧАНЕ НА НОВИНИ (от news source):")
print("-" * 80)

try:
    from src.data.news_loader import fetch_yahoo_finance_news
    
    # Извличаме новини за AAPL
    print("Извличане на новини за AAPL от Yahoo Finance...")
    news_list = fetch_yahoo_finance_news('AAPL', limit=3)
    
    if news_list:
        print(f"✅ Извлечени {len(news_list)} новини")
        print("\\nПримерна новина:")
        news_item = news_list[0]
        print(f"  Title: {news_item['title']}")
        print(f"  Publisher: {news_item['publisher']}")
        print(f"  Time: {news_item['published_time']}")
        
        # Стъпка 2: Обработка с FinBERT
        print("\\n\\n2. ОБРАБОТКА С FINBERT:")
        print("-" * 80)
        
        from src.models.news_encoder import FinBERTEncoder
        encoder = FinBERTEncoder()
        
        # Взимаме текста на новината
        news_text = news_item['title']  # Може да добавим и description/summary
        
        print(f"Обработване на новина: '{news_text[:80]}...'")
        
        # Получаваме embeddings
        embeddings = encoder.encode_text(news_text)
        print(f"\\n✅ Embeddings shape: {embeddings.shape}")
        print(f"   (Това са {embeddings.shape[0]} числа които ще използваме като features)")
        
        # Получаваме sentiment
        sentiment = encoder.get_sentiment(news_text)
        print(f"\\n✅ Sentiment: {sentiment['label']}")
        print(f"   Probabilities: {sentiment['probabilities']}")
        
        print("\\n\\n3. РЕЗУЛТАТ:")
        print("-" * 80)
        print("""
        ✅ Имаме новини (от Yahoo Finance)
        ✅ Имаме embeddings (от FinBERT) - 768 числа
        ✅ Имаме sentiment (от FinBERT) - positive/negative/neutral
        
        Следваща стъпка: Добавяне на embeddings към техническите features
        и трениране на модела!
        """)
        
    else:
        print("⚠️  Не са намерени новини. Може да е проблем с yfinance.")
        
except Exception as e:
    print(f"❌ Грешка: {e}")
    print("\\nТова е нормално ако yfinance не е инсталиран.")
    print("Инсталирай с: pip install yfinance")

print("\\n" + "=" * 80)
print("ЗАКЛЮЧЕНИЕ:")
print("=" * 80)
print("""
✅ News Source (yfinance) → Извлича новини от интернет
✅ FinBERT → Обработва новините и ги конвертира в embeddings
✅ Our Model → Използва embeddings като features за прогнози

FinBERT е инструмент за обработка, не източник на новини!
""")

## Решение за исторически новини (2010-2024)

За целия период на модела (2010-2024) имаме няколко опции:

In [ ]:
# Опции за исторически новини (2010-2024)

print("=" * 80)
print("ОПЦИИ ЗА ИСТОРИЧЕСКИ НОВИНИ (2010-2024)")
print("=" * 80)

options = {
    "1. Yahoo Finance RSS Feeds": {
        "Покритие": "Ограничено (обикновено последните 1-2 години)",
        "Безплатно": "✅ Да",
        "API Key": "❌ Не",
        "Трудност": "Лесно",
        "Препоръка": "За тестване, но не за пълно покритие"
    },
    "2. Web Scraping (Yahoo Finance, Seeking Alpha)": {
        "Покритие": "Пълно (може да се стигне до 2010)",
        "Безплатно": "✅ Да",
        "API Key": "❌ Не",
        "Трудност": "Средно-трудно",
        "Препоръка": "Най-добро решение за безплатни данни"
    },
    "3. Alpha Vantage News API": {
        "Покритие": "Ограничено (последните 1-2 години)",
        "Безплатно": "✅ Да (free tier)",
        "API Key": "✅ Да (безплатен)",
        "Трудност": "Лесно",
        "Препоръка": "Допълнение към scraping"
    },
    "4. Financial Modeling Prep API": {
        "Покритие": "Добро (няколко години назад)",
        "Безплатно": "⚠️ Частично (free tier ограничен)",
        "API Key": "✅ Да",
        "Трудност": "Лесно",
        "Препоръка": "Ако free tier е достатъчен"
    },
    "5. Pre-collected Datasets": {
        "Покритие": "Зависи от dataset-а",
        "Безплатно": "✅ Да (ако намерим такъв)",
        "API Key": "❌ Не",
        "Трудност": "Много лесно",
        "Препоръка": "Най-добро ако намерим подходящ dataset"
    },
}

for option, details in options.items():
    print(f"\\n{option}:")
    for key, value in details.items():
        print(f"  {key}: {value}")

print("\\n" + "=" * 80)
print("ПРЕПОРЪКА:")
print("=" * 80)
print("""
За целия период (2010-2024) препоръчвам КОМБИНИРАН подход:

1. **Web Scraping** (основен източник)
   - Yahoo Finance news pages (с уважение към rate limits)
   - Seeking Alpha (ако има RSS или разрешение)
   - MarketWatch (ако е достъпен)
   
2. **RSS Feeds** (допълнение)
   - Yahoo Finance RSS
   - Google News RSS (филтрирано за финанси)
   
3. **API-та** (за последните години)
   - Alpha Vantage (за sentiment scores)
   - yfinance (за най-новите новини)

ВАЖНО:
- Винаги проверявай robots.txt и Terms of Service
- Добавяй забавяния между заявките (1-2 секунди)
- Кеширай данните за да не правиш повторни заявки
- Започни с малък тест преди пълно извличане

Следваща стъпка: Създаване на scraping модул с правилно handling на rate limits.
""")

In [4]:
# Check Hugging Face dataset structure
from datasets import load_dataset_builder, get_dataset_infos

dataset_name = config.data.dataset_name
print(f"Checking dataset: {dataset_name}")

try:
    builder = load_dataset_builder(dataset_name)
    print(f"\nDataset info:")
    print(f"  Description: {builder.info.description[:200]}..." if builder.info.description else "  No description")
    print(f"  Features: {builder.info.features}")
    
    # Check for news-related features
    if hasattr(builder.info, 'features'):
        feature_names = list(builder.info.features.keys()) if isinstance(builder.info.features, dict) else []
        news_features = [f for f in feature_names if any(kw in f.lower() for kw in ['news', 'article', 'headline', 'text'])]
        
        if news_features:
            print(f"\n✅ Found news-related features: {news_features}")
        else:
            print(f"\n❌ No news-related features found")
            print(f"\nAvailable features: {feature_names[:20]}")
except Exception as e:
    print(f"\n⚠️  Could not load dataset builder: {e}")
    print("This is okay - we'll proceed with external news sources")

Checking dataset: pmoe7/SP_500_Stocks_Data-ratios_news_price_10_yrs


Repo card metadata block was not found. Setting CardData to empty.



Dataset info:
  No description
  Features: {'_id': Value('string'), 'compound': Value('float64'), 'date': Value('string'), 'headline': Value('string'), 'neg': Value('float64'), 'neu': Value('float64'), 'pos': Value('float64'), 'ticker': Value('string'), 'time': Value('string')}

✅ Found news-related features: ['headline']


## 1.3 Test News APIs

Let's test different news APIs to see which one works best for our use case.

### Option 1: Alpha Vantage News API

In [ ]:
# Alpha Vantage News API
# Documentation: https://www.alphavantage.co/documentation/#news-sentiment
# Free tier: 5 API calls per minute, 500 calls per day

import requests
import os
from datetime import datetime, timedelta

# You'll need to get an API key from https://www.alphavantage.co/support/#api-key
ALPHA_VANTAGE_API_KEY = os.getenv('GKXZ5NT78PUAR2DT', 'demo')  # Replace with your key

def test_alpha_vantage_news(ticker='AAPL', limit=10):
    """Test Alpha Vantage News API"""
    url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'NEWS_SENTIMENT',
        'tickers': ticker,
        'limit': limit,
        'apikey': ALPHA_VANTAGE_API_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if 'Information' in data:
            print(f"⚠️  API Info: {data['Information']}")
            return None
        
        if 'feed' in data:
            articles = data['feed']
            print(f"✅ Successfully fetched {len(articles)} articles for {ticker}")
            
            if articles:
                print(f"\nSample article:")
                article = articles[0]
                print(f"  Title: {article.get('title', 'N/A')[:100]}...")
                print(f"  Source: {article.get('source', 'N/A')}")
                print(f"  Time: {article.get('time_published', 'N/A')}")
                print(f"  Sentiment Score: {article.get('overall_sentiment_score', 'N/A')}")
                print(f"  Relevance Score: {article.get('relevance_score', 'N/A')}")
            
            return articles
        else:
            print(f"❌ Unexpected response format: {list(data.keys())}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("Testing Alpha Vantage News API...")
print("Note: You need to set ALPHA_VANTAGE_API_KEY environment variable")
print("Get free API key at: https://www.alphavantage.co/support/#api-key\n")

if ALPHA_VANTAGE_API_KEY != 'demo':
    alpha_vantage_results = test_alpha_vantage_news('AAPL', limit=5)
else:
    print("⚠️  Using demo key - will likely hit rate limit. Set your API key to test properly.")

Testing Alpha Vantage News API...
Note: You need to set ALPHA_VANTAGE_API_KEY environment variable
Get free API key at: https://www.alphavantage.co/support/#api-key

⚠️  Using demo key - will likely hit rate limit. Set your API key to test properly.


### Option 2: NewsAPI

In [10]:
# NewsAPI
# Documentation: https://newsapi.org/docs
# Free tier: 100 requests per day

NEWSAPI_KEY = os.getenv('NEWSAPI_KEY', '')  # Get from https://newsapi.org/register

def test_newsapi(ticker='AAPL', days_back=7):
    """Test NewsAPI"""
    if not NEWSAPI_KEY:
        print("⚠️  NEWSAPI_KEY not set. Get one at https://newsapi.org/register")
        return None
    
    url = 'https://newsapi.org/v2/everything'
    
    # Calculate date range
    to_date = datetime.now()
    from_date = to_date - timedelta(days=days_back)
    
    params = {
        'q': ticker,  # Search query
        'from': from_date.strftime('%Y-%m-%d'),
        'to': to_date.strftime('%Y-%m-%d'),
        'sortBy': 'publishedAt',
        'language': 'en',
        'pageSize': 10,
        'apiKey': NEWSAPI_KEY
    }
    
    try:
        response = requests.get(url, params=params, timeout=10)
        data = response.json()
        
        if data.get('status') == 'ok':
            articles = data.get('articles', [])
            print(f"✅ Successfully fetched {len(articles)} articles for {ticker}")
            
            if articles:
                print(f"\nSample article:")
                article = articles[0]
                print(f"  Title: {article.get('title', 'N/A')[:100]}...")
                print(f"  Source: {article.get('source', {}).get('name', 'N/A')}")
                print(f"  Published: {article.get('publishedAt', 'N/A')}")
                print(f"  Description: {article.get('description', 'N/A')[:150]}...")
            
            return articles
        else:
            print(f"❌ Error: {data.get('message', 'Unknown error')}")
            return None
            
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

print("Testing NewsAPI...")
if NEWSAPI_KEY:
    newsapi_results = test_newsapi('AAPL', days_back=7)
else:
    print("⚠️  Set NEWSAPI_KEY environment variable to test")

Testing NewsAPI...
⚠️  Set NEWSAPI_KEY environment variable to test


### Option 3: Yahoo Finance (Free, no API key needed)

In [9]:
# Yahoo Finance - Free option using yfinance library
# No API key needed, but may have rate limits

try:
    import yfinance as yf
    
    def test_yahoo_finance_news(ticker='AAPL'):
        """Test Yahoo Finance news"""
        try:
            stock = yf.Ticker(ticker)
            news = stock.news
            
            if news:
                print(f"✅ Successfully fetched {len(news)} news items for {ticker}")
                
                if news:
                    print(f"\nSample news item:")
                    item = news[0]
                    print(f"  Title: {item.get('title', 'N/A')[:100]}...")
                    print(f"  Publisher: {item.get('publisher', 'N/A')}")
                    print(f"  Published: {item.get('providerPublishTime', 'N/A')}")
                    print(f"  Link: {item.get('link', 'N/A')[:80]}...")
                
                return news
            else:
                print(f"⚠️  No news found for {ticker}")
                return None
                
        except Exception as e:
            print(f"❌ Error: {e}")
            return None
    
    print("Testing Yahoo Finance (yfinance)...")
    yahoo_results = test_yahoo_finance_news('AAPL')
    
except ImportError:
    print("⚠️  yfinance not installed. Install with: pip install yfinance")
    print("This is a free option with no API key needed!")

Testing Yahoo Finance (yfinance)...
✅ Successfully fetched 10 news items for AAPL

Sample news item:
  Title: N/A...
  Publisher: N/A
  Published: N/A
  Link: N/A...


## 1.4 Comparison and Recommendation

In [11]:
# Create comparison table
comparison_data = {
    'Source': ['Alpha Vantage', 'NewsAPI', 'Yahoo Finance (yfinance)'],
    'API Key Required': ['Yes (free)', 'Yes (free)', 'No'],
    'Rate Limit': ['5/min, 500/day', '100/day', 'Unknown (may vary)'],
    'Historical Data': ['Limited', 'Limited (1 month free)', 'Recent only'],
    'Sentiment Score': ['Yes (built-in)', 'No (need separate)', 'No'],
    'Cost': ['Free tier available', 'Free tier available', 'Free'],
    'Ease of Use': ['Medium', 'Easy', 'Very Easy'],
}

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("NEWS SOURCES COMPARISON")
print("="*80)
print(df_comparison.to_string(index=False))

print("\n" + "="*80)
print("RECOMMENDATION")
print("="*80)
print("""
For Phase 1, I recommend:

1. **Start with Yahoo Finance (yfinance)** - Easiest to set up, no API key needed
   - Good for testing and prototyping
   - May have limitations for historical data

2. **Alpha Vantage as backup** - If we need sentiment scores built-in
   - Requires API key but free tier is generous
   - Has built-in sentiment analysis

3. **For production/historical data** - May need paid service or web scraping
   - Consider Financial Modeling Prep or Polygon.io for better historical coverage

Next steps:
- Test yfinance with multiple tickers
- Check if we can get historical news (may need to scrape or use paid API)
- Evaluate data quality and coverage
""")


NEWS SOURCES COMPARISON
                  Source API Key Required         Rate Limit        Historical Data    Sentiment Score                Cost Ease of Use
           Alpha Vantage       Yes (free)     5/min, 500/day                Limited     Yes (built-in) Free tier available      Medium
                 NewsAPI       Yes (free)            100/day Limited (1 month free) No (need separate) Free tier available        Easy
Yahoo Finance (yfinance)               No Unknown (may vary)            Recent only                 No                Free   Very Easy

RECOMMENDATION

For Phase 1, I recommend:

1. **Start with Yahoo Finance (yfinance)** - Easiest to set up, no API key needed
   - Good for testing and prototyping
   - May have limitations for historical data

2. **Alpha Vantage as backup** - If we need sentiment scores built-in
   - Requires API key but free tier is generous
   - Has built-in sentiment analysis

3. **For production/historical data** - May need paid service or web

## 1.5 Test Historical News Availability

In [12]:
# Test if we can get historical news (important for backtesting)
print("Testing historical news availability...")
print("\nNote: Most free APIs only provide recent news (last 1-30 days)")
print("For backtesting, we need news from 2010-2024 period")
print("\nOptions:")
print("1. Use paid API with historical data (Financial Modeling Prep, Polygon.io)")
print("2. Web scraping from financial news sites (with proper attribution)")
print("3. Use pre-collected datasets (like the Hugging Face one if it has news)")
print("4. Start with recent data and expand backwards gradually")

# Check if yfinance can get older news
try:
    import yfinance as yf
    from datetime import datetime
    
    stock = yf.Ticker('AAPL')
    news = stock.news
    
    if news:
        # Check date range
        dates = []
        for item in news:
            if 'providerPublishTime' in item:
                ts = item['providerPublishTime']
                dt = datetime.fromtimestamp(ts)
                dates.append(dt)
        
        if dates:
            oldest = min(dates)
            newest = max(dates)
            print(f"\n✅ Yahoo Finance news date range:")
            print(f"   Oldest: {oldest.strftime('%Y-%m-%d')}")
            print(f"   Newest: {newest.strftime('%Y-%m-%d')}")
            print(f"   Coverage: {(newest - oldest).days} days")
            
            if oldest.year < 2020:
                print(f"\n✅ Good! Can get historical data back to {oldest.year}")
            else:
                print(f"\n⚠️  Limited historical data - only goes back to {oldest.year}")
                print(f"   May need alternative source for full backtest period")
except Exception as e:
    print(f"\n⚠️  Could not test historical availability: {e}")

Testing historical news availability...

Note: Most free APIs only provide recent news (last 1-30 days)
For backtesting, we need news from 2010-2024 period

Options:
1. Use paid API with historical data (Financial Modeling Prep, Polygon.io)
2. Web scraping from financial news sites (with proper attribution)
3. Use pre-collected datasets (like the Hugging Face one if it has news)
4. Start with recent data and expand backwards gradually


## Summary and Next Steps

In [ ]:
print("\n" + "="*80)
print("PHASE 1 SUMMARY")
print("="*80)
print("""
✅ Completed:
1. Checked current dataset structure
2. Tested multiple news API options
3. Compared different sources

📋 Next Steps (Phase 2):
1. Choose primary news source (recommendation: yfinance for start)
2. Create news_loader.py module
3. Test fetching news for multiple tickers
4. Design storage format for news data
5. Create script to fetch historical news

💡 Key Decisions:
- Primary source: Yahoo Finance (yfinance) - easiest to start
- Backup: Alpha Vantage (if we need sentiment scores)
- Historical data: May need paid API or web scraping for full coverage
""")